In [1]:
# import sys
# sys.path.append('home/Documents/code/github/motile_tracker/') #maybe needed for debugging??


import numpy as np
import pandas as pd
import dask.array as da
import matplotlib.pyplot as plt
import napari
import shutil

from pathlib import Path
# from typing import Dict, List, Tuple, Optional, Iterable,TypeAlias, Mapping, Sequence
# from pyinstrument import Profiler

from trackedit.motile_overwrites import *
# from trackedit.DatabaseArray import DatabaseArray
# from functions.get_value_by_field import get_value_by_field
from trackedit.TrackEdit_functions import *
from trackedit.DatabaseHandler import DatabaseHandler

from motile_tracker.data_model.actions import AddEdges, DeleteNodes, DeleteEdges, AddNodes


Make segments+graph

In [2]:
#**********INPUTS*********
working_directory = Path('/home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/')
db_filename_old = 'data.db'
data_shape_full = [851,73,1024,1024]      #T,(Z),Y,X       (851,73,1024,1024)
scale = (4,1,1)
layer_name = 'ultrack'
allow_overwrite = True      #overwrite existing database/changelog
#*************************

DB_handler = DatabaseHandler(
                db_filename_old = db_filename_old,
                working_directory = working_directory,
                data_shape_full = data_shape_full,
                z_scale = scale[0],
                name = 'ultrack',
                allow_overwrite = allow_overwrite)

#overwrite some motile functions
DeleteNodes._apply = create_db_delete_nodes(DB_handler)
DeleteEdges._apply = create_db_delete_edges(DB_handler)
AddEdges._apply = create_db_add_edges(DB_handler)
AddNodes._apply = create_db_add_nodes(DB_handler)
TracksViewer._refresh = create_tracks_viewer_and_segments_refresh(layer_name=layer_name)

Database copied to: /home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/data_v1.db
old database: data.db
new database: data_v1.db
new logfile: data_v1_changelog.txt


TrackEdit

In [3]:
viewer = napari.Viewer()

trackeditclass = TrackEditClass(viewer,
                                databasehandler = DB_handler)

viewer.dims.ndisplay = 3    #3D view

Making new tracking view controller


In [4]:
# CHECK THE NUMBER OF TRACKS IN TRACKS LIST 









